In [1]:
#@ First lets check the number of classes in PubMEdQA
from datasets import load_dataset

ds = load_dataset("qiaojin/PubMedQA", "pqa_artificial")

print(ds)

README.md:   0%|          | 0.00/5.19k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/233M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/211269 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['pubid', 'question', 'context', 'long_answer', 'final_decision'],
        num_rows: 211269
    })
})


In [2]:
labels = set(ds['train']['final_decision'])
labels

{'no', 'yes'}

Okay so number of classes = $2$

In [3]:
#@ Step 1: Clone the github repo 
!git clone https://github.com/Firojpaudel/GEM.git

#@ Step 2: Install all requirements 
!pip install  -r /kaggle/working/GEM/requirements.txt -qq

Cloning into 'GEM'...
remote: Enumerating objects: 44, done.
remote: Counting objects: 100% (44/44), done.
remote: Compressing objects: 100% (33/33), done.
remote: Total 44 (delta 22), reused 25 (delta 9), pack-reused 0 (from 0)
Receiving objects: 100% (44/44), 15.42 KiB | 3.85 MiB/s, done.
Resolving deltas: 100% (22/22), done.


In [4]:
import warnings 
warnings.filterwarnings('ignore')

In [5]:
#@ Step 3: Add repo to path
import sys
sys.path.append('/kaggle/working/GEM')

#@ Step 4: Import and run function
from gem_trainer import run_gem_pipeline
from datasets import load_dataset

#@ Rest of the code as above
dataset_subset = ds['train'].select(range(20000))
print(dataset_subset)

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Dataset({
    features: ['pubid', 'question', 'context', 'long_answer', 'final_decision'],
    num_rows: 20000
})


In [6]:
### Train-test splittooooo
dataset = dataset_subset.train_test_split(test_size=0.2, seed=42) ## ahh splitting nowww

In [7]:
##@ Label Map..
label_map = {'yes': 0, 'no': 1}

In [8]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

In [9]:
##@ Redefining the tokenization before calling the pipeline function
def tokenize_fn(examples):
    # Combine 'question' and 'context['contexts']' into a single string per example
    inputs = [q + " [SEP] " + " ".join(c['contexts']) for q, c in zip(examples['question'], examples['context'])]
    
    # Map 'final_decision' to numerical labels
    labels = [label_map[label] for label in examples['final_decision']]
    
    # Tokenize the combined inputs
    tokenized = tokenizer(
        inputs,
        padding='max_length',
        truncation=True,
        max_length=128
    )
    
    # Add labels to the tokenized output
    tokenized['labels'] = labels
    return tokenized

In [10]:
## Pipeline Call
results = run_gem_pipeline(
    dataset=dataset,
    model_name="bert-base-uncased",
    num_classes=2,
    num_epochs=5,
    batch_size=128,
    learning_rate=2e-5,
    max_seq_length=128,
    gradient_accum_steps=2,
    cluster_size=256,
    threshold=0.65,
    tokenize_fn=tokenize_fn,
    save_path="gem_model_pubmedqa_final.pth",  
    checkpoint_dir="checkpoints",     
    checkpoint_interval=2            
)

Map:   0%|          | 0/16000 [00:00<?, ? examples/s]

Map:   0%|          | 0/4000 [00:00<?, ? examples/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
100%|██████████| 125/125 [20:39<00:00,  9.91s/it]


Epoch 1/5 | Avg Loss: 0.4772


...

Epoch 4/5 | Avg Loss: 0.3999
Saved checkpoint to checkpoints/checkpoint_epoch_4.pth


100%|██████████| 125/125 [20:29<00:00,  9.83s/it]


Epoch 5/5 | Avg Loss: 0.3862


100%|██████████| 32/32 [03:56<00:00,  7.40s/it]


Final Accuracy: 92.58%
Saved final model to gem_model_pubmedqa_final.pth
